## Biogas plant

You want to plan the two-year supply of raw materials for a biogas power plant. Such a plant produces energy by burning biogas, which is obtained from the bacterial fermentation of organic wastes. 
Specifically, your plant is powered by corn chopping, a residual of agro-industrial operations that you can purchase from 5 local farms. 
The table below shows the quarterly capacity of each farm for the next two years. Quantities are measured in tons.

Farm|T1|T2|T3|T4|T5|T6|T7|T8
:-|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:
1|700|1500|700|0|0|700|1500|0
2|1350|0|450|0|1350|0|450|0
3|0|1500|1500|0|0|1500|1500|0
4|820|1560|820|0|820|1560|820|0
5|0|680|1080|0|0|680|1080|0

Due to crop rotations and corn harvesting periods, farms are unable to supply material in some quarters. Moreover the types of corn chopping provided are different, each coming with its own unitary purchase price, unitary storage cost and percentage of dry matter. The table below shows a summary of these information.

Farm|Purchase price|Storage cost|Dry matter
:-|:-:|:-:|:-:
1|0.20|0.002|15
2|0.18|0.012|28
3|0.19|0.007|35
4|0.21|0.011|37
5|0.23|0.015|42

Your biogas plant must operate by burning a mixture of corn choppings with a dry matter percentage between 20% and 40%. Under these conditions, the yield is 421.6 kWh of energy per ton of burned material. The energy produced by the plant is sold on the market at a price of 0.28 $/kWh. 

Due to state regulations, all biogas plants can produce a maximum of 1950 MWh of energy per quarter. You are allowed to store corn chopping in a silo, whose total capacity is of 500 tons. 

Plan the supply and inventory of your biogas plant with the goal of maximizing your profits (i.e., revenues minus costs).

In [8]:
# When using Colab, make sure you run this instruction beforehand
!pip install --upgrade cffi==1.15.0
import importlib
import cffi
importlib.reload(cffi)
!pip install mip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
import mip

A = [[700, 1500, 700, 0, 0, 700, 1500, 0],
     [1350, 0, 450, 0, 1350, 0, 450, 0],
     [0, 1500, 1500, 0, 0, 1500, 1500, 0],
     [820, 1560, 820, 0, 820, 1560, 820, 0],
     [0, 680, 1080, 0, 0, 680, 1080, 0]]

# A summarize the availability of corn of each farm for each quarter.


B = [[0.20, 0.002, 15],
     [0.18, 0.012, 28],
     [0.19, 0.007, 35],
     [0.21, 0.011, 37],
     [0.23, 0.015, 42]]    

# B[0] is the cost of buying corn.
# B[1] is the cost of storing corn.
# B[2] is the dry matter percentage of each type of corn.

n_farms = len(A)
n_quarters = len(A[0])

min_dry = 20
max_dry = 40

energy = 421.6

revenue = 0.28

max_energy = 1950

max_store = 500

# n_farms and n_quarters are the number of farms and quarters. Of course they directly depends from the dimension of matrix A.
# min_dry and max_dry are the minimum and maximum dry percentage required to burn a mixture of corn.
# energy is the amount of energy (kW) produced per ton. 
# revenue is the revenue for each kW of biogas sold.
# max_energy is the maximum MW of energy that can be produced each quarter. 
# max_store is the maximum amount of tons which can be stored in silos each quarter.

m = mip.Model()

x = [[m.add_var() for i in range(n_quarters)] for j in range(n_farms)]

# x is a variable which represents how many tons are bought from each farm each quarter.

y = [[m.add_var() for i in range(n_quarters)] for j in range(n_farms)]

# y is a variable which represents how many tons are stored of each type of corn each quarter.

# x[i] - y[i] + y[i-1] represents how many tons are burned and sold during quarter i (i > 0). It's the sum of corn bought during quarter i and corn stored during quarter i-1
# minus corn stored during quarter i.

total_revenues = (mip.xsum(x[i][j] for i in range(n_farms) for j in range(n_quarters - 1)) - mip.xsum(y[i][7] for i in range(n_farms))) * energy * revenue

# total_revenues is the total of all revenues during two years. It's the sum of all tons bought from each farm subtracted the tons which remain in silos at last quarter.
# Infact this difference is the amount of burned tons. Then this amount is multiplied for 421.6 which is the amount of energy per ton of burned corn and again for 0.28 
# which is the revenue per kW of biogas sold.

total_costs = mip.xsum(x[i][j] * B[i][0] for i in range(n_farms) for j in range(n_quarters)) + mip.xsum(y[i][j] * B[i][1] for i in range(n_farms) for j in range(n_quarters))

# total_costs is the total of all costs during two years. It's the sum of all tons bought times their price and the sum of all tons stored times their price.

m.objective = mip.maximize (total_revenues - total_costs)

# the objective is to maximize the profit.

for j in range(n_quarters):

  for i in range(n_farms):
    m.add_constr(x[i][j] <= A[i][j])
    if j > 0:
      m.add_constr(y[i][j] <= (x[i][j] + y[i][j-1]))
    else:
      m.add_constr(y[i][j] <= x[i][j])
    m.add_constr(x[i][j] >= 0)
    m.add_constr(y[i][j] >= 0)

    # the constraints avoid buying more corns than farm's availability of each quarter and storing more corns than bought.

  if j > 0:
    burned_per_quarter = mip.xsum(x[i][j] + y[i][j-1] - y[i][j] for i in range(n_farms))
    m.add_constr((burned_per_quarter * energy) / 1000 <= max_energy)

    # burned_per_quarter is the tons of energy burned each quarter. After first quarter it's the amount of tons bought minus tons of corn stored during current quarter 
    # plus the amount of stored tons during previous quarter. The constrains avoid that this amount times energy (421.6) divided by 1000 (from kW to MW) is less than max_energy (1950).
    
    dry_matter = mip.xsum((x[i][j] + y[i][j-1] - y[i][j]) * B[i][2] for i in range(n_farms))
    m.add_constr(dry_matter >= (min_dry * burned_per_quarter))
    m.add_constr(dry_matter <= (max_dry * burned_per_quarter))

    # dry_matter is the total amount of dry matter burned each quarter. It's the sum of burned corn times their dry matter percentage.
    # the constrains avoid that this amount is greater than total burned corn times min_dry (which means is greater than 20%) and less than total burned corn
    # times max_dry (which means less than 40%).

  else:
    burned_per_quarter = mip.xsum(x[i][j] - y[i][j] for i in range(n_farms))
    m.add_constr((burned_per_quarter * energy) / 1000 <= max_energy)

    # burned_per_quarter is the tons of energy burned. In the first quarter it's only the amount of tons bought minus tons of corn stored during current quarter.
    # the constrains are to avoid that this amount times energy (421.6) divided by 1000 (from kW to MW) is less than max_energy (1950).

    dry_matter = mip.xsum((x[i][j] - y[i][j]) * B[i][2] for i in range(n_farms))
    m.add_constr(dry_matter >= (min_dry * burned_per_quarter))
    m.add_constr(dry_matter <= (max_dry * burned_per_quarter))

  stored_per_quarter = mip.xsum(y[i][j] for i in range(n_farms))
  m.add_constr(stored_per_quarter <= max_store)

  # stored_per_quarter is the tons of corns stored each quarter. The contrains are to avoid storing more corn than maximum (500)

m.optimize()
print(f"PROFIT: {(total_revenues - total_costs).x}")


PROFIT: 2861373.9254127145
